In [244]:
import os
import re
import numpy as np
import pandas as pd
from urllib.request import urlopen
import wget
from bs4 import BeautifulSoup
import PyPDF2
import camelot
import tabula
import datetime
import matplotlib.pyplot as plt

In [260]:
def extract_df_from_pdf(filename):
    
    reader = PyPDF2.PdfFileReader(open(filename, "rb"))
    
    # Extract the date from the 1st page and store it:
    page = reader.getPage(0)
    text = page.extractText()
    text_tolist = text.split()
    months = ["January", "February", "March", "April", "May", "June", "July"]
    for month in months:
        if month in text_tolist:
            idx = text_tolist.index(month)
    d = text_tolist[idx-1]
    m = text_tolist[idx].replace("January","01").replace("February","02").replace("March","03").replace("April","04").replace("May","05").replace("June","06")
    y = text_tolist[idx+1].replace("*","")
    date = datetime.date(int(y), int(m), int(d))
    
    # As pdf scraping is tricky, the packages tabula and camelot are tried. 
    # If tabula fails, it tries camelot. 
    # If both fail, the function will return no dataframe at all.
    try:
        tables = tabula.read_pdf(filename, pages = "all", multiple_tables = True)
        print(date,"tabula")
    except:
        try:
            tables_camelot = camelot.read_pdf(filename, 
                              pages='all', 
                              flavour = 'stream',
                              line_scale=60, shift_text=[''], 
                              flag_size=True)#,
                              #split_text=True)#, 
                              #strip_text='.\n')
            # camelot returns a list of objects that have the property df, 
            # therefore tables[0].df is the first dataframe.
            # We'll create a list of dataframes, which is simpler and 
            # it matches with what tabula does:
            tables = []
            for table in tables_camelot:
                tables.append(table.df)
            print(date,"camelot")
        except:
            return

    # Identify the LAST table that contains the appropriate headers. 
    # I'll look for the words 'country', 'case' AND 'death'.
    for it, table in enumerate(tables):
        text = table.to_string().casefold()
        list_words = ["country", "case", "death", "new"]
        res = [word for word in list_words if word in text]
        if len(res) == len(list_words):
            itable = it
    columns = tables[itable].columns
        
    # The names of the columns change from one pdf to other
    # Here we identify the columns corresponding to the 
    # text strings in the array list_str.
    # Then, change their original name to what is in the array.
    # This way, all the tables will have the same name for the columns. 
    # It is important that 'new confirmed' comes before 'confirmed'!
    columns_clean = ["Country", "New conf", "Conf", "New deaths", "Deaths"]
    list_idx = []
    list_idx_clean = []
    for idx, col in enumerate(columns):
        for idx_clean, str_clean in enumerate(columns_clean):
            if idx_clean not in list_idx_clean:
                match = []
                for word in str_clean.split():
                    text_column = tables[itable].iloc[:,idx].to_string().casefold()
                    if word == "Country":
                        match.append(word.casefold() in text_column)
                    else:
                        match.append(word.casefold() in text_column and "china" not in text_column)
                if match != [] and all(match) and idx not in list_idx:
                    list_idx.append(idx)
                    list_idx_clean.append(idx_clean)

    idx_country = list_idx[0]
    
    # Append data to list "tot":
    countries = ["China", "Spain", "Korea", "Italy", "Germany", "United States of America"]
    tot = []
    for idx_table, table in enumerate(tables):
        dd = table
        if len(dd.columns) > idx_country:# and idx_table >= itable:
            for country in countries:
                # Look for the name of the country in the corresponding column:
                try:
                    #x = dd.iloc[:,idx_country].str.contains(country)==True
                    x = dd.iloc[:,idx_country].str.casefold() == country.casefold()
                    if x.any():
                        tot.append(dd[x].to_numpy().ravel())
                except:
                    x = []

    # Assign columns:
    dft = pd.DataFrame(data=tot).iloc[:,list_idx]
    dft.columns = [columns_clean[i] for i in list_idx_clean]
    dft["Date"] = date
    
    # Fill any empty gap with NaN:
    dft = dft.replace(r'^\s*$', np.nan, regex=True)
    #dft2 = dft2.replace(r'^ *$', "_", regex=True)

    # Split columns if necessary:
    str = "New conf"
    x = dft[str].str.contains("\(")
    if x.any():
        dft[["Conf", str]] = dft[str].str.split(pat="\(", expand=True).fillna(value=0)
        dft[["Deaths", "New deaths"]] = dft["New deaths"].str.split(pat="\(", expand=True).fillna(value=0)
    
    # Clean data
    # Remove parenthesis, HTML tags and non-alphabet chars:
    dft = dft.replace("\(","",regex=True).replace("\)","",regex=True)
    dft = dft.replace("<.*?>","",regex=True).replace("([^\s\w]|_)+","",regex=True)
    
    # Remove white spaces in numeric columns:
    #dft2.select_dtypes(include=['int']) =  dft2.select_dtypes(include=['int']).replace(" ","")
    dft[["Conf", "New conf", "Deaths", "New deaths"]] = dft[["Conf", "New conf", "Deaths", "New deaths"]].replace("\s+","",regex=True)
    # Remove rows containing the word "development":
    for intruder in ["development", "spread"]:
        dft = dft[~dft.Country.str.contains(intruder)]

    return dft

In [261]:
url0 = "https://www.who.int"
url = url0 + "/emergencies/diseases/novel-coronavirus-2019/situation-reports/"
html = urlopen(url)
soup = BeautifulSoup(html, features="html.parser")

In [262]:
# From all the href links, select those which contain the word "Situation":
links = [a for a in soup.findAll('a', href=True) if "Situation" in a.text]

In [263]:
print(len(links))

141


In [264]:
list_df = []
for link in links:
    wfile = url0 + link["href"]
    ff = wfile.rsplit('/', 1)[-1].split('?',1)[0]
    print(ff)
    if not os.path.isfile(ff):
        filename = wget.download(wfile)
    try:
        df = extract_df_from_pdf(ff)
    except:
        print("Error!")
    #except ValueError:
    #    print("could not be read properly")
    #except IndexError:
    #    print("IndexError!")
    list_df.append(df)

20200610-covid-19-sitrep-142.pdf
2020-06-10 tabula
20200609-covid-19-sitrep-141.pdf
2020-06-09 tabula
20200608-covid-19-sitrep-140.pdf
2020-06-08 tabula
20200607-covid-19-sitrep-139.pdf
2020-06-07 tabula
20200606-covid-19-sitrep-138.pdf
2020-06-06 tabula
20200605-covid-19-sitrep-137.pdf
2020-06-05 tabula
20200604-covid-19-sitrep-136.pdf
2020-06-04 tabula
20200603-covid-19-sitrep-135.pdf
2020-06-03 tabula
20200602-covid-19-sitrep-134.pdf
2020-06-02 tabula
20200601-covid-19-sitrep-133.pdf
2020-06-01 tabula
20200531-covid-19-sitrep-132.pdf
2020-05-31 tabula
20200530-covid-19-sitrep-131.pdf
2020-05-30 tabula
20200529-covid-19-sitrep-130.pdf
2020-05-09 tabula
20200528-covid-19-sitrep-129.pdf
2020-05-08 tabula
20200527-covid-19-sitrep-128.pdf
2020-05-07 tabula
20200526-covid-19-sitrep-127.pdf
2020-05-06 tabula
20200525-covid-19-sitrep-126.pdf
2020-05-05 tabula
20200524-covid-19-sitrep-125.pdf
2020-05-04 tabula
20200523-covid-19-sitrep-124.pdf
2020-05-03 tabula
20200522-covid-19-sitrep-123.pd

In [265]:
print(len(list_df))

141


In [266]:
final = pd.concat(list_df, ignore_index=True)

In [267]:
pd.unique(final["Date"])

array([datetime.date(2020, 6, 10), datetime.date(2020, 6, 9),
       datetime.date(2020, 6, 8), datetime.date(2020, 6, 7),
       datetime.date(2020, 6, 6), datetime.date(2020, 6, 5),
       datetime.date(2020, 6, 4), datetime.date(2020, 6, 3),
       datetime.date(2020, 6, 2), datetime.date(2020, 6, 1),
       datetime.date(2020, 5, 31), datetime.date(2020, 5, 30),
       datetime.date(2020, 5, 9), datetime.date(2020, 5, 8),
       datetime.date(2020, 5, 7), datetime.date(2020, 5, 6),
       datetime.date(2020, 5, 5), datetime.date(2020, 5, 4),
       datetime.date(2020, 5, 3), datetime.date(2020, 5, 2),
       datetime.date(2020, 5, 21), datetime.date(2020, 5, 20),
       datetime.date(2020, 5, 12), datetime.date(2020, 5, 11),
       datetime.date(2020, 5, 10), datetime.date(2020, 5, 1),
       datetime.date(2020, 4, 30), datetime.date(2020, 4, 29),
       datetime.date(2020, 4, 27), datetime.date(2020, 4, 26),
       datetime.date(2020, 4, 25), datetime.date(2020, 4, 24),
       dat

In [268]:
final.to_csv("corona-who-14.csv", sep=",")